In [1]:
# creation of a simple regression model with the mean of each variable in datasets_3h folder
# and the mean of the target variable in the training set

import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [6]:
# test 2

# open each csv file and read it into a pandas dataframe
name_of_variables = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets/variable_list_24_v2_1.csv')
name_of_variables = name_of_variables['variable'].tolist()

# Create a new list of variables
new_name_of_variables = []
for variable in name_of_variables:
    if variable == 'geopotential':
        new_name_of_variables.extend(['geopotential_500', 'geopotential_1000'])
    else:
        new_name_of_variables.append(variable)

# Iterate over the new list of variables
for i in range(0, len(new_name_of_variables)):

    print(new_name_of_variables[i])

    locals()[f'max_{new_name_of_variables[i]}'] = pd.DataFrame()
    locals()[f'min_{new_name_of_variables[i]}'] = pd.DataFrame()
    locals()[f'mean_{new_name_of_variables[i]}'] = pd.DataFrame()
    locals()[f'sigma_{new_name_of_variables[i]}'] = pd.DataFrame()

    for j in range (1,96+1):
        if 'geopotential' in new_name_of_variables[i]:
            for k in [500, 1000]:
                df_max_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/geopotential/storm_' + str(j) + '/max_'+ str(j) + '_' + str(k) + '.csv')
                df_min_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/geopotential/storm_' + str(j) + '/min_'+ str(j) + '_' + str(k) + '.csv')
                df_mean_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/geopotential/storm_' + str(j) + '/mean_'+ str(j) + '_' + str(k) + '.csv')
                df_sigma_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/geopotential/storm_' + str(j) + '/std_'+ str(j) + '_' + str(k) + '.csv')

        else:
            df_max_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/' + new_name_of_variables[i] + '/storm_' + str(j) + '/max_'+ str(j) + '_0.csv')
            df_min_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/' + new_name_of_variables[i] + '/storm_' + str(j) + '/min_'+ str(j) + '_0.csv')
            df_mean_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/' + new_name_of_variables[i] + '/storm_' + str(j) + '/mean_'+ str(j) + '_0.csv')
            df_sigma_temp = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project/datasets_3h/' + new_name_of_variables[i] + '/storm_' + str(j) + '/std_'+ str(j) + '_0.csv')

        df_max_temp = df_max_temp.drop(columns = ['Unnamed: 0'])
        df_min_temp = df_min_temp.drop(columns = ['Unnamed: 0'])
        df_mean_temp = df_mean_temp.drop(columns = ['Unnamed: 0'])
        df_sigma_temp = df_sigma_temp.drop(columns = ['Unnamed: 0'])

        locals()[f'max_{new_name_of_variables[i]}'] = pd.concat([locals()[f'max_{new_name_of_variables[i]}'], df_max_temp], axis=0)
        locals()[f'min_{new_name_of_variables[i]}'] = pd.concat([locals()[f'min_{new_name_of_variables[i]}'], df_min_temp], axis=0)
        locals()[f'mean_{new_name_of_variables[i]}'] = pd.concat([locals()[f'mean_{new_name_of_variables[i]}'], df_mean_temp], axis=0)
        locals()[f'sigma_{new_name_of_variables[i]}'] = pd.concat([locals()[f'sigma_{new_name_of_variables[i]}'], df_sigma_temp], axis=0)

10m_u_component_of_wind
10m_v_component_of_wind
2m_dewpoint_temperature
2m_temperature
cloud_base_height
convective_available_potential_energy
convective_inhibition
convective_precipitation
convective_rain_rate
convective_snowfall
geopotential_500
geopotential_1000
high_cloud_cover
instantaneous_10m_wind_gust
k_index
large_scale_precipitation
large_scale_snowfall
mean_large_scale_precipitation_rate
mean_top_net_long_wave_radiation_flux
mean_top_net_short_wave_radiation_flux
mean_total_precipitation_rate
mean_sea_level_pressure
mean_surface_latent_heat_flux
mean_surface_net_long_wave_radiation_flux
mean_surface_net_short_wave_radiation_flux
mean_vertically_integrated_moisture_divergence
surface_pressure
total_precipitation
total_totals_index


In [7]:
name_of_variables = new_name_of_variables

# remove variable with only one repetitive value in each statistic
for i in range (0, len(name_of_variables)):
    var_name = name_of_variables[i]
    for stat in ['max', 'min', 'mean', 'sigma']:
        df = locals()[f'{stat}_{var_name}']
        cols_to_drop = [col for col in df.columns if df[col].nunique() <= 1]
        locals()[f'{stat}_{var_name}'] = df.drop(columns=cols_to_drop)

# drop nan values and remove empty dataframes
kept_variables = []
for i in range (0, len(name_of_variables)):
    var_name = name_of_variables[i]
    for stat in ['max', 'min', 'mean', 'sigma']:
        df = locals()[f'{stat}_{var_name}']
        df = df.dropna()
        if not df.empty:
            locals()[f'{stat}_{var_name}'] = df
            if var_name not in kept_variables:
                kept_variables.append(var_name)
        else:
            del locals()[f'{stat}_{var_name}']

'''del max_high_cloud_cover
del min_high_cloud_cover'''

# update name_of_variables to only include kept variables
name_of_variables = pd.Series(kept_variables)

In [30]:
# the target variable is the mean of instanteneous_10m_wind_gust
variable_w_high_corr= ['mean_large_scale_precipitation', 'mean_total_precipitation','mean_mean_large_scale_precipitation_rate','mean_large_scale_snowfall']
# mean_low_tropospheric_vertical_velocity??
# create a dataframe with the target variable

target_variable = mean_instantaneous_10m_wind_gust
x_variable = mean_total_precipitation

# split the data into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(x_variable, target_variable, test_size=0.2, random_state=42)

In [32]:
for x_var in variable_w_high_corr:
    X_train, X_test, y_train, y_test = train_test_split(locals()[f'{x_var}'], target_variable, test_size=0.2, random_state=42)
    # create a linear regression model
    model = LinearRegression()

    # fit the model to the training data
    model.fit(X_train, y_train)

    # make predictions on the test set
    y_pred = model.predict(X_test)

    # calculate the mean squared error of the model
    mse = mean_squared_error(y_test, y_pred)
    print(f'The mean squared error of the model with {x_var} as the independent variable is {mse}')

The mean squared error of the model with mean_large_scale_precipitation as the independent variable is 0.06319021937398077
The mean squared error of the model with mean_total_precipitation as the independent variable is 0.06923697814313828
The mean squared error of the model with mean_mean_large_scale_precipitation_rate as the independent variable is 0.0633238140785473
The mean squared error of the model with mean_large_scale_snowfall as the independent variable is 0.07187408704374666


In [24]:
# train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [25]:
# make predictions
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(mse)

0.06923697814313828


In [26]:
# standardize the data
scaler = StandardScaler()
scaler.fit(X_train)
scaler.fit(y_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

# train the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(mse)

0.7751150756182278


In [33]:
for x_var in variable_w_high_corr:
    X_train, X_test, y_train, y_test = train_test_split(locals()[f'{x_var}'], target_variable, test_size=0.2, random_state=42)
    # standardize the data
    scaler = StandardScaler()
    scaler.fit(X_train)
    scaler.fit(y_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)

    # train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # make predictions
    y_pred = model.predict(X_test)

    # calculate the mean squared error
    mse = mean_squared_error(y_test, y_pred)
    print(f'The mean squared error of the model with {x_var} as the independent variable is {mse}')

The mean squared error of the model with mean_large_scale_precipitation as the independent variable is 0.7074209906708323
The mean squared error of the model with mean_total_precipitation as the independent variable is 0.7751150756182278
The mean squared error of the model with mean_mean_large_scale_precipitation_rate as the independent variable is 0.7089165971062966
The mean squared error of the model with mean_large_scale_snowfall as the independent variable is 0.8046377803937472
